In [7]:
# import requests
# url = 'https://www.metacritic.com/browse/movies/score/metascore/year/filtered?sort=desc'
# headers = {
#     'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}

# page = requests.get(url, headers=headers, timeout=5)
# page

In [36]:
# to help bypass 403 and 429 error codes
from urllib.request import Request, urlopen

# for scraping
from bs4 import BeautifulSoup as BS

In [37]:
url = 'https://www.metacritic.com/browse/movies/score/metascore/year/filtered?sort=desc'

# bypass errors using Request/urlopen changing user-agent
headers = {'User-Agent': 'Firefox'}
req = Request(url, headers=headers)

page = urlopen(req).read()
page

b'<!DOCTYPE html>\n<html xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://ogp.me/ns/fb#">\n<head>\n            <title>Best Movies for 2019 - Metacritic</title>\n        \n    <meta http-equiv="content-type" content="text/html; charset=UTF-8"><script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHandler","finished","addToTrace","inlineHit","addRelease"],d="api-",l=d+"ixn-";a(p,

In [38]:
# beautifulsoup object "soup" of webpage
soup = BS(page, 'html.parser')
# print(soup.prettify())

In [63]:
movies = soup.find_all('td', class_='clamp-summary-wrap')
movies

[<td class="clamp-summary-wrap">
 <input class="clamp-summary-expand" id="550625" type="checkbox">
 <div class="clamp-score-wrap">
 <a class="metascore_anchor" href="/movie/parasite/critic-reviews">
 <div class="metascore_w large movie positive">95</div>
 </a>
 </div>
 <span class="title numbered">
                                                                             1.
                                                                     </span>
 <a class="title" href="/movie/parasite">
 <h3>Parasite</h3>
 </a>
 <div class="clamp-details">
 <span>October 11, 2019</span>
 <span class="cert_rating Not Rated"> | Not Rated</span>
 </div>
 <div class="summary">
                             Meet the Park Family: the picture of aspirational wealth. And the Kim Family, rich in street smarts but not much else. Be it chance or fate, these two houses are brought together and the Kims sense a golden opportunity. Masterminded by college-aged Ki-woo, the Kim children expediently install thems

In [96]:
movie_keys = ['title', 'release_date', 'metascore']

# loop through list of 100 movies and pull relevant data

movie_titles = []
movie_dates = []
movie_metascores = []
movie_tuples = []

for movie in movies:
    
    movie_title = movie.find('h3').text
    movie_titles.append(movie_title)
    
    movie_date = movie.find('div', class_='clamp-details').span.text
    movie_dates.append(movie_date)
    
    movie_metascore = movie.find('div', class_='metascore_w large movie positive').text
    movie_metascores.append(movie_metascore)

    # add details to a tuple to add to MYSQL db
    movie_tuple = (movie_title, movie_date, movie_metascore)
    movie_tuples.append(movie_tuple)

# Does not account for missing data (2nd movie's user score is a Nan value)
#     movie_userscore = movie.find('div', class_='metascore_w user large movie positive').text
#     print(movie_userscore)

movie_tuples



[('Parasite', 'October 11, 2019', '95'),
 ('Marriage Story', 'November 6, 2019', '95'),
 ('The Irishman', 'November 1, 2019', '94'),
 ('Portrait of a Lady on Fire', 'December 6, 2019', '93'),
 ('Apocalypse Now: Final Cut', 'August 15, 2019', '92'),
 ('The Souvenir', 'May 17, 2019', '92'),
 ('Uncut Gems', 'December 13, 2019', '91'),
 ('For Sama', 'July 26, 2019', '90'),
 ('The Farewell', 'July 12, 2019', '89'),
 ("Long Day's Journey Into Night", 'April 12, 2019', '89'),
 ('Pain and Glory', 'October 4, 2019', '88'),
 ('Apollo 11', 'March 1, 2019', '88'),
 ('Genèse (Genesis)', 'August 23, 2019', '87'),
 ('In Fabric', 'December 6, 2019', '87'),
 ('Diane', 'March 29, 2019', '86'),
 ('An Elephant Sitting Still', 'March 8, 2019', '86'),
 ('Rolling Thunder Revue: A Bob Dylan Story by Martin Scorsese',
  'June 12, 2019',
  '86'),
 ('The Wild Pear Tree', 'January 30, 2019', '86'),
 ('American Factory', 'August 21, 2019', '86'),
 ('Black Mother', 'March 8, 2019', '86'),
 ('Honeyland', 'July 26, 2

In [102]:
import config
from mysql.connector import errorcode

In [100]:
## connecting to the database using 'connect()' method

cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

cursor = cnx.cursor()

In [104]:
db_name = 'Movies'

In [105]:
# create table in MYSQL

TABLES = {}
TABLES['Metascore'] = """
                CREATE TABLE Metascore 
                (
                    title varchar(50) NOT NULL UNIQUE,
                    release_date date NOT NULL,
                    metascore int(10) NOT NULL,
                    PRIMARY KEY (title)
                );
                """

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table Metascore: No database selected


In [ ]:
cursor.close()
cnx.close()